#### Návrh RF filtru 

Pásmová propust s následujícími požadovanými parametry: 

##### Aproximace 	
Značení parametrů stejné jako v programu Syntfil https://asinus.feld.cvut.cz/syntfil

Pásmová propust, Chebyshev, 3. řád
f-s = 142.551MHz, f-p = 143.025MHz, f_p = 143.075MHz, fs = 143.551MHz, fm = 143.05MHz, Δfp = 50kHz, Δfs = 1MHz, ap = 3dB, as = 60dB

##### LC realizace
zakončení: symetrické, R = 50Ω, směr: zepředu, struktura: π 

Pro optimalizaci se používá nástroj https://github.com/ahkab/ahkab
